# Gaia Distance of the Open Cluster Pleiades
Gaia is an European Space Agency (ESA) space mission that aims at measuring the position and parallax of 1 billion stars in our galaxy by revisiting each field 70 times over the mission lifetime. It will also provide radial velocity measurements for the brightest 150 million stars. For more about Gaia see https://sci.esa.int/web/gaia/-/28820-summary

Pleiades(M45), also known as the Seven Sisters, is an open star cluster that is visible to eyes in the night sky. It one of the closest star clusters. However, the distance to Pleiades has been controversial, because Gaia's predecessor, Hipparcos, measured a distance that was in disagreement with almost all other methods:
<img src="pleiades.jpg" width=500x>

People have previously thought that the disagreement had to do with our lack of understanding of the physics of stars in this cluster. However, with more recent radio Very Long Baseline Interferometry (VLBI) measurements and new data from Gaia, it seems like the Hipparcos measurement was off for reasons that we still do not know. Léo Girardi offered a good summary in his Perspective piece on the Science issue that published the VLBI results (https://science.sciencemag.org/content/345/6200/1001).

In this exercise, you will get data from the Gaia catalog, isolate members of the Pleiades from other stars in the field, and measure the distance to the cluster!

## Getting the data

Some imports. Also define a utility class so that you get an error when you forget to fill in code segments.

If you see `REPLACE_WITH_YOUR_SOLUTION()`, replace it with your own code unless otherwise directed.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
class SolutionMissingError(Exception):
    def __init__(self, text=None):
        Exception.__init__(self, text if text else "You need to complete the solution for this code to work!")
def REPLACE_WITH_YOUR_SOLUTION(words=None):
    raise SolutionMissingError(words)

Some configuration stuff. If you're connected to the ethernet, set the following to `True`. Point `CSV_FILE_PATH` to where the downloaded data file is.

In [ ]:
HAS_GOOD_INTERNET = False
CSV_FILE_PATH = './query.ecsv'


If you have good internet, we can use astroquery to explore the Gaia catalog, and download the data toward Pleiades (M45).

We use an asynchronous query (asynchronous rather than synchronous queries should be performed when retrieving more than 2000 rows) centerd on the Pleiades (coordinates: 56.75, +24.1167) with a search radius of 2 degrees and save the results to a file.

Note: The query to the archive is with ADQL (Astronomical Data Query Language). For a description of ADQL and more examples see the Gaia DR1 ADQL cookbook: https://gaia.ac.uk/data/gaia-data-release-1/adql-cookbook.

If the internet is not good. You can point `CSV_FILE_PATH` from the above cell to the downloaded data.

In [ ]:
if HAS_GOOD_INTERNET:
    from astroquery.gaia import Gaia
    job = Gaia.launch_job_async("SELECT * \
        FROM gaiadr2.gaia_source \
        WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',56.75,24.1167,2))=1;" \
        , dump_to_file=True)
    table = job.get_results()
else:
    from astropy.table import Table
    table = Table.read(CSV_FILE_PATH, format='ascii.ecsv')

Can you make sense of the query that we just launched? In particular, what does ICRS mean?

In [ ]:
REPLACE_WITH_YOUR_SOLUTION("Talk to your partner about the query. No coding required here")

Now we look at the column names. Not all of these are the most transparent things ever. For a description of all the columns see https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html. **You will need to refer to the list of columns for the next part of this exercise**

In [ ]:
table.colnames

In [ ]:
table['solution_id'].shape

Our data contain 110203 stars!

## Finding the cluster
The most common way to find a cluster is to look for stars that move together. We can start by scatter plotting the proper motion in RA and DEC.

If we were to do this with Gaia's Data Release 1 (DR1), we would be seeing a cluster of points in this plane corresponding to the Pleiades. However, there are many more stars in the Data Release 2 (DR2), and we need to do more work to identify the cluster from all the stars in the field.

In [ ]:
plt.scatter(table['pmra'], table['pmdec'], color='r', alpha=0.3)
plt.xlabel('PMRA(mas/year)')
plt.ylabel('PMDEC(mas/year)')
plt.xlim(-60,80)
plt.ylim(-120,30)

We don't see the cluster :(. You will have to come up with a set of the criteria so that the cluster would pop up (somewhat). You might want to consider the phot_g_mean_mag, parallax, parallax_error, pmra_error, and pmde_error.
You can use `plt.hist()` to get a sense of how those parameters are distributed.

Note that the Pleiades is relatively close by. So Gaia should measure a pretty significant parallax for it.

While you're at it. How would you look for one of those hypervelocity stars from exploded white dwarf binaries that we explored in Problem Set 1?

In [ ]:
# Explore the distributions of the data here. Use hist and scatter to zero in on the cluster.
plt.hist(table['parallax'])

In [ ]:
REPLACE_WITH_YOUR_SOLUTION()
# Edit the following function to apply your selection criteria for members of the cluster
def find_cluster(table):
    # this is just an example that illustrates the syntax. It is not likely to be super useful.
    m45_table = table[
        (table['pmra_error']<table['pmra']) & \
    (table['pmdec_error']<table['pmdec'])
    ]
    return m45_table

In [ ]:
# Check your result
m45_table = find_cluster(table)
plt.scatter(m45_table['pmra'], m45_table['pmdec'], color='r', alpha=0.3)
plt.xlabel('PMRA(mas/year)')
plt.ylabel('PMDEC(mas/year)')
plt.xlim(-60,80)
plt.ylim(-120,30)

In [ ]:
# how many stars do we have left?
m45_table['solution_id'].shape

## The HR Diagram
Since all the stars in a cluster are roughly born at the same time, you should be able to plot an observer's HR diagram (color-magnitude), with appropriately chosen data columns. Can you spot the main sequence?

Now correct the magnitude by the distance modulus, see how the HR diagram changes. Compare it to the H-R diagram obtained by Gaia, how is this different from the Gaia HR Diagram? What do these differences tell you about the cluster?
<img src="gaia-hr.jpg">

In [ ]:
# Make an H-R diagram! Make sure you get the directions of the axes right.
# You might find plt.gca().invert_xaxis() and plt.gca().invert_yaxis() useful
REPLACE_WITH_YOUR_SOLUTION()

Now correct the magnitude by the distance modulus, see how the HR diagram changes. Compare it to the H-R diagram obtained by Gaia, how is this different from the Gaia HR Diagram? What do these differences tell you about the cluster?
<img src="gaia-hr.jpg">

In [ ]:
# Make an H-R diagram! Make sure you get the directions of the axes right.
# You might find plt.gca().invert_xaxis() and plt.gca().invert_yaxis() useful
REPLACE_WITH_YOUR_SOLUTION()

Are there other quantities in the Gaia catalog that can be plotted in a HR diagram? Try taking a different set of quantities and see what you get.

In [ ]:
REPLACE_WITH_YOUR_SOLUTION()

## Now onto the distance
Talk to your partner to come up with at least two ways of measuring the distance to the cluster, given the data.

Now execute the simplest one. No measurements or inferences are good without uncertainty. Come up with a procedure to determine the error in your measurement.

In [ ]:
REPLACE_WITH_YOUR_SOLUTION()

## Epilogue
Getting a distance from a parallax might seem like straightforward. But measurements have uncertainties, and properly describing the uncertainties and converting the uncertainties in parallax measurements into distances is not always the most straightforward thing ever, because distance is not a linear function of parallax. I would recommend reading the Introduction section of Bailer-Jones etl al. 2018 (https://arxiv.org/abs/1804.10121) to get a sense of the nuances involved in inferring distances from parallax measurements.